# Student Data Exploration

## Universidad Autónoma de Chile - Canvas LMS Analytics

This notebook explores **individual student data** from Canvas LMS.

### Available Endpoints for Student Data

| Endpoint | Description |
|----------|-------------|
| `/api/v1/users/{id}` | Basic user profile |
| `/api/v1/users/{id}/enrollments` | All course enrollments |
| `/api/v1/users/{id}/page_views` | Clickstream activity |
| `/api/v1/courses/{cid}/analytics/users/{uid}/activity` | Per-course activity |
| `/api/v1/courses/{cid}/enrollments` | Grades (filter by user) |

---

In [ ]:
# =============================================================================
# SETUP & IMPORTS
# =============================================================================

import sys
import os
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from IPython.display import display, HTML
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Add scripts directory to path
scripts_dir = os.path.join(os.path.dirname(os.getcwd()), 'scripts')
if scripts_dir not in sys.path:
    sys.path.insert(0, scripts_dir)

# Import discovery module
from discovery import CanvasClient, PASS_THRESHOLD

# Initialize client
client = CanvasClient()

# Check API status
quota, status = client.check_quota()
print(f"API Status: {quota} - {status}")
print(f"Pass Threshold: {PASS_THRESHOLD}%")
print("\n✓ Setup complete")

## 1. Student Configuration

Set the student `user_id` to analyze. You can find student IDs from:
- Course enrollments list
- Previous notebook analyses
- Canvas admin interface

In [ ]:
# =============================================================================
# STUDENT CONFIGURATION
# =============================================================================

STUDENT_USER_ID = 117656  # <- CHANGE THIS TO YOUR TARGET STUDENT ID

print(f"Target Student ID: {STUDENT_USER_ID}")
print("="*60)

## 2. Student Profile

Fetch basic student information from `/api/v1/users/{id}`

In [ ]:
# =============================================================================
# STUDENT PROFILE
# =============================================================================
# Endpoint: /api/v1/users/{id}
# =============================================================================

print("="*60)
print("STUDENT PROFILE")
print("="*60)
print(f"Endpoint: /api/v1/users/{STUDENT_USER_ID}")
print("-"*60)

# Fetch user profile
user_response = client.safe_get(
    f"{client.api_url}/api/v1/users/{STUDENT_USER_ID}"
)

if user_response:
    user_data = user_response.json()
    
    STUDENT_PROFILE = {
        'user_id': user_data.get('id'),
        'name': user_data.get('name', 'N/A'),
        'sortable_name': user_data.get('sortable_name', 'N/A'),
        'short_name': user_data.get('short_name', 'N/A'),
        'login_id': user_data.get('login_id', 'N/A'),
        'email': user_data.get('email', 'N/A'),
        'created_at': user_data.get('created_at', 'N/A'),
        'time_zone': user_data.get('time_zone', 'N/A'),
        'locale': user_data.get('locale', 'N/A'),
    }
    
    df_profile = pd.DataFrame([
        {'Field': k.replace('_', ' ').title(), 'Value': v}
        for k, v in STUDENT_PROFILE.items()
    ])
    
    display(df_profile.style.hide(axis='index').set_caption("Student Profile"))
    
    print(f"\n📋 SUMMARY: Student '{STUDENT_PROFILE['name']}' (ID: {STUDENT_PROFILE['user_id']})")
else:
    print(f"[ERROR] Could not fetch user {STUDENT_USER_ID}")
    STUDENT_PROFILE = {}

## 3. Student Enrollments

Fetch all course enrollments for this student from `/api/v1/users/{id}/enrollments`

In [ ]:
# =============================================================================
# STUDENT ENROLLMENTS
# =============================================================================
# Endpoint: /api/v1/users/{id}/enrollments
# =============================================================================

print("="*60)
print("STUDENT ENROLLMENTS")
print("="*60)
print(f"Endpoint: /api/v1/users/{STUDENT_USER_ID}/enrollments")
print("-"*60)

# Fetch enrollments with grades
enrollments = client.paginate(
    f"{client.api_url}/api/v1/users/{STUDENT_USER_ID}/enrollments",
    params={'include[]': ['grades', 'current_grading_period_scores'], 'per_page': 100}
)

print(f"\nTotal Enrollments: {len(enrollments)}")

if len(enrollments) > 0:
    # Parse enrollments
    enrollment_data = []
    for e in enrollments:
        grades = e.get('grades', {})
        enrollment_data.append({
            'Course ID': e.get('course_id'),
            'Course Name': e.get('course_integration_id', 'N/A'),  # May not be available
            'Role': e.get('role', 'N/A'),
            'State': e.get('enrollment_state', 'N/A'),
            'Current Score': grades.get('current_score'),
            'Final Score': grades.get('final_score'),
            'Current Grade': grades.get('current_grade', 'N/A'),
            'Last Activity': e.get('last_activity_at', 'N/A')[:16] if e.get('last_activity_at') else 'N/A',
            'Total Activity (min)': round((e.get('total_activity_time') or 0) / 60, 1),
        })
    
    df_enrollments = pd.DataFrame(enrollment_data)
    
    # Filter to student enrollments only
    df_student_enrollments = df_enrollments[df_enrollments['Role'].str.contains('Student', case=False, na=False)]
    
    print(f"\n[3.1] All Enrollments:")
    display(df_enrollments.style.hide(axis='index').format({
        'Current Score': '{:.1f}',
        'Final Score': '{:.1f}',
    }, na_rep='N/A'))
    
    # Summary statistics
    print(f"\n[3.2] Enrollment Summary:")
    print(f"   Total Enrollments: {len(df_enrollments)}")
    print(f"   As Student: {len(df_student_enrollments)}")
    
    # State breakdown
    states = df_enrollments['State'].value_counts()
    print(f"\n   By State:")
    for state, count in states.items():
        print(f"      {state}: {count}")
    
    # Courses with grades
    with_grades = df_enrollments['Current Score'].notna().sum()
    print(f"\n   Courses with Grades: {with_grades}")
    
    if with_grades > 0:
        avg_score = df_enrollments['Current Score'].mean()
        print(f"   Average Score: {avg_score:.1f}%")
    
    STUDENT_ENROLLMENTS = df_enrollments
    
    print(f"\n📋 SUMMARY: Student enrolled in {len(df_enrollments)} courses, {with_grades} with grades")
else:
    print("   No enrollments found.")
    STUDENT_ENROLLMENTS = pd.DataFrame()

## 4. Detailed Course Information

Get detailed information about each course the student is enrolled in.

In [ ]:
# =============================================================================
# DETAILED COURSE INFORMATION
# =============================================================================

print("="*60)
print("DETAILED COURSE INFORMATION")
print("="*60)
print("-"*60)

if len(STUDENT_ENROLLMENTS) > 0:
    # Get unique course IDs
    course_ids = STUDENT_ENROLLMENTS['Course ID'].dropna().unique()
    
    print(f"\nFetching details for {len(course_ids)} courses...")
    
    course_details = []
    for cid in course_ids[:20]:  # Limit to first 20 courses
        try:
            course = client.get_course(int(cid))
            if course:
                # Get student's grade in this course
                student_enrollment = STUDENT_ENROLLMENTS[
                    STUDENT_ENROLLMENTS['Course ID'] == cid
                ].iloc[0] if len(STUDENT_ENROLLMENTS[STUDENT_ENROLLMENTS['Course ID'] == cid]) > 0 else {}
                
                course_details.append({
                    'Course ID': course.get('id'),
                    'Course Name': course.get('name', 'N/A')[:40],
                    'Term': course.get('term', {}).get('name', 'N/A')[:25] if course.get('term') else 'N/A',
                    'Total Students': course.get('total_students', 0),
                    'Student Score': student_enrollment.get('Current Score') if isinstance(student_enrollment, dict) else student_enrollment['Current Score'] if 'Current Score' in student_enrollment else None,
                    'State': course.get('workflow_state', 'N/A'),
                })
        except Exception as e:
            print(f"   Error fetching course {cid}: {e}")
    
    if len(course_details) > 0:
        df_courses = pd.DataFrame(course_details)
        
        print(f"\n[4.1] Course Details:")
        display(df_courses.style.hide(axis='index').format({
            'Student Score': '{:.1f}%',
        }, na_rep='N/A'))
        
        STUDENT_COURSES = df_courses
        
        print(f"\n📋 SUMMARY: Retrieved details for {len(df_courses)} courses")
    else:
        print("   Could not fetch course details.")
        STUDENT_COURSES = pd.DataFrame()
else:
    print("   No enrollments to fetch course details for.")
    STUDENT_COURSES = pd.DataFrame()

## 5. Activity Summary

Basic activity metrics for the student across all courses.

In [ ]:
# =============================================================================
# ACTIVITY SUMMARY
# =============================================================================

print("="*60)
print("ACTIVITY SUMMARY")
print("="*60)
print("-"*60)

if len(STUDENT_ENROLLMENTS) > 0:
    # Calculate total activity from enrollments
    total_activity_min = STUDENT_ENROLLMENTS['Total Activity (min)'].sum()
    courses_with_activity = (STUDENT_ENROLLMENTS['Total Activity (min)'] > 0).sum()
    
    print(f"\n[5.1] Overall Activity:")
    print(f"   Total Activity Time: {total_activity_min:.1f} minutes ({total_activity_min/60:.1f} hours)")
    print(f"   Courses with Activity: {courses_with_activity}")
    print(f"   Avg Activity per Course: {total_activity_min/len(STUDENT_ENROLLMENTS):.1f} minutes")
    
    # Most active courses
    print(f"\n[5.2] Most Active Courses (by time):")
    top_activity = STUDENT_ENROLLMENTS.nlargest(5, 'Total Activity (min)')[
        ['Course ID', 'Total Activity (min)', 'Current Score']
    ]
    display(top_activity.style.hide(axis='index').format({
        'Total Activity (min)': '{:.1f}',
        'Current Score': '{:.1f}%'
    }, na_rep='N/A'))
    
    # Last activity dates
    print(f"\n[5.3] Recent Activity:")
    recent = STUDENT_ENROLLMENTS[STUDENT_ENROLLMENTS['Last Activity'] != 'N/A'].nlargest(5, 'Last Activity')[
        ['Course ID', 'Last Activity', 'Current Score']
    ]
    if len(recent) > 0:
        display(recent.style.hide(axis='index').format({
            'Current Score': '{:.1f}%'
        }, na_rep='N/A'))
    else:
        print("   No recent activity data available.")
    
    STUDENT_ACTIVITY_SUMMARY = {
        'total_activity_min': total_activity_min,
        'courses_with_activity': courses_with_activity,
        'avg_per_course': total_activity_min/len(STUDENT_ENROLLMENTS) if len(STUDENT_ENROLLMENTS) > 0 else 0,
    }
    
    print(f"\n📋 SUMMARY: {total_activity_min:.1f} minutes total activity across {courses_with_activity} courses")
else:
    print("   No enrollment data for activity summary.")
    STUDENT_ACTIVITY_SUMMARY = {}

## 6. Student Overview Summary

Consolidated view of all student information.

In [ ]:
# =============================================================================
# STUDENT OVERVIEW SUMMARY
# =============================================================================

print("="*70)
print("STUDENT OVERVIEW SUMMARY")
print("="*70)

# Get metrics
name = STUDENT_PROFILE.get('name', 'N/A')
user_id = STUDENT_PROFILE.get('user_id', STUDENT_USER_ID)
total_courses = len(STUDENT_ENROLLMENTS) if 'STUDENT_ENROLLMENTS' in dir() else 0
courses_with_grades = STUDENT_ENROLLMENTS['Current Score'].notna().sum() if 'STUDENT_ENROLLMENTS' in dir() and len(STUDENT_ENROLLMENTS) > 0 else 0
avg_score = STUDENT_ENROLLMENTS['Current Score'].mean() if 'STUDENT_ENROLLMENTS' in dir() and len(STUDENT_ENROLLMENTS) > 0 else 0
total_activity = STUDENT_ACTIVITY_SUMMARY.get('total_activity_min', 0) if 'STUDENT_ACTIVITY_SUMMARY' in dir() else 0

# Pass/Fail status
if courses_with_grades > 0:
    passing_courses = (STUDENT_ENROLLMENTS['Current Score'] >= PASS_THRESHOLD).sum()
    failing_courses = ((STUDENT_ENROLLMENTS['Current Score'] < PASS_THRESHOLD) & (STUDENT_ENROLLMENTS['Current Score'].notna())).sum()
else:
    passing_courses = 0
    failing_courses = 0

print(f"""
┌─────────────────────────────────────────────────────────────────────┐
│ STUDENT: {name:<56} │
│ User ID: {user_id:<56} │
├─────────────────────────────────────────────────────────────────────┤
│                                                                     │
│ ENROLLMENT SUMMARY                                                  │
│ ──────────────────                                                  │
│   Total Courses Enrolled:     {total_courses:<35} │
│   Courses with Grades:        {courses_with_grades:<35} │
│   Average Score:              {f'{avg_score:.1f}%' if avg_score else 'N/A':<35} │
│                                                                     │
│ ACADEMIC STATUS                                                     │
│ ───────────────                                                     │
│   Passing (>={PASS_THRESHOLD}%):            {passing_courses:<35} │
│   Failing (<{PASS_THRESHOLD}%):             {failing_courses:<35} │
│                                                                     │
│ ACTIVITY                                                            │
│ ────────                                                            │
│   Total Activity Time:        {f'{total_activity:.1f} min ({total_activity/60:.1f} hrs)':<35} │
│                                                                     │
└─────────────────────────────────────────────────────────────────────┘
""")

# Store complete student data
STUDENT_DATA = {
    'user_id': user_id,
    'profile': STUDENT_PROFILE,
    'enrollments': STUDENT_ENROLLMENTS if 'STUDENT_ENROLLMENTS' in dir() else pd.DataFrame(),
    'courses': STUDENT_COURSES if 'STUDENT_COURSES' in dir() else pd.DataFrame(),
    'activity_summary': STUDENT_ACTIVITY_SUMMARY if 'STUDENT_ACTIVITY_SUMMARY' in dir() else {},
    'metrics': {
        'total_courses': total_courses,
        'courses_with_grades': courses_with_grades,
        'avg_score': avg_score,
        'passing_courses': passing_courses,
        'failing_courses': failing_courses,
        'total_activity_min': total_activity,
    }
}

print("\n✓ All student data stored in STUDENT_DATA variable")
print(f"\nAnalysis completed: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

---

## Next Steps

Future sections to add:

- [ ] **Page Views Analysis** - Detailed clickstream data
- [ ] **Per-Course Deep Dive** - Submissions, assignments, quizzes
- [ ] **Time-of-Day Patterns** - When does this student study?
- [ ] **Grade Trajectory** - How are grades changing over time?
- [ ] **Peer Comparison** - How does this student compare to classmates?
- [ ] **Risk Assessment** - Is this student at risk of failing?
